In [18]:
%pip install nltk


     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.5 MB 544.7 kB/s eta 0:00:03
     -- ------------------------------------- 0.1/1.5 MB 521.8 kB/s eta 0:00:03
     ----- ---------------------------------- 0.2/1.5 MB 980.4 kB/s eta 0:00:02
     ------- -------------------------------- 0.3/1.5 MB 1.1 MB/s eta 0:00:02
     ------------ --------------------------- 0.5/1.5 MB 1.6 MB/s eta 0:00:01
     ------------------ --------------------- 0.7/1.5 MB 2.1 MB/s eta 0:00:01
     ----------------------------- ---------- 1.1/1.5 MB 2.8 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   --

In [3]:
%pip install scikit-learn
%pip install flask-cors

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK resources (you only need to do this once)
# nltk.download("punkt")
# nltk.download("stopwords")

# Function to load dream objects from a JSON file
def load_dream_objects(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Function to preprocess and tokenize text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Stem the words
    stemmer = SnowballStemmer("english")
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return " ".join(stemmed_tokens)

# Function to find related dream objects using NLTK
def find_related_dreams(user_input, dream_objects):
    related_dreams = []
    user_input = preprocess_text(user_input)

    for dream in dream_objects:
        dream_text = preprocess_text(dream["typeOfDream"])

        # Calculate cosine similarity between user input and dream text
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform([user_input, dream_text])
        similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

        # You can adjust this threshold to control the sensitivity of the search
        if similarity > 0.3:  # Example threshold; adjust as needed
            related_dreams.append(dream)
    return related_dreams

# User input
user_input = input("Enter your dream description: ")

# Load dream objects from the JSON file
dream_objects = load_dream_objects("dreams.json")

# Find related dream objects using NLTK
related_dreams = find_related_dreams(user_input, dream_objects)

# Print the list of related dream objects
for dream in related_dreams:
    print(dream)
print('Program finished')

# You can also format the output as a JSON string if needed.


{'typeOfDream': 'Being chased by a lion', 'prediction': 'This dream indicates the presence of challenges or obstacles in your life. It may suggest that you are facing or will face some form of danger or difficult situation. It could represent a sense of fear or insecurity in your waking life.', 'thingsToDoIfHaveToAvoidIt': 'To avoid potential negative outcomes, it is advisable to approach situations with caution and be prepared for any obstacles that may arise. Focus on building inner strength and courage to overcome challenges. Seek support from trusted individuals who can offer guidance and assistance in difficult times.'}
Program finished


GET API for the above code : install flask

In [2]:
%pip install flask

   ---------------------------------------- 0.0/99.7 kB ? eta -:--:--
   ------------ --------------------------- 30.7/99.7 kB 1.3 MB/s eta 0:00:01
   -------------------------------- ------- 81.9/99.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 99.7/99.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/226.6 kB ? eta -:--:--
   --------------------- ------------------ 122.9/226.6 kB 3.6 MB/s eta 0:00:01
   ------------------------------------ --- 204.8/226.6 kB 2.5 MB/s eta 0:00:01
   ---------------------------------------- 226.6/226.6 kB 2.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import json
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)
CORS(app)
print('Libs loaded')
# Preload dream objects from a JSON file
with open('dreams.json', 'r') as file:
    dream_objects = json.load(file)
print(str(len(dream_objects))+ ' dreams loaded')
# Download NLTK resources (you only need to do this once)
# nltk.download("punkt")
# nltk.download("stopwords")

# Function to preprocess and tokenize text
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

    # Stem the words
    stemmer = SnowballStemmer("english")
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

    return " ".join(stemmed_tokens)

# Function to find related dream objects using NLTK
def find_related_dreams(user_input, dream_objects):
    related_dreams = []
    user_input = preprocess_text(user_input)

    for dream in dream_objects:
        dream_text = preprocess_text(dream["typeOfDream"])

        # Calculate cosine similarity between user input and dream text
        tfidf_vectorizer = TfidfVectorizer()
        tfidf_matrix = tfidf_vectorizer.fit_transform([user_input, dream_text])
        similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

        # You can adjust this threshold to control the sensitivity of the search
        if similarity > 0.3:  # Example threshold; adjust as needed
            related_dreams.append(dream)
    return related_dreams

@app.route('/find_related_dreams', methods=['GET'])
@cross_origin()
def get_related_dreams():
    user_input = request.args.get('user_input')
    if not user_input:
        return jsonify({"error": "Please provide user input as a query parameter."}), 400

    # Find related dream objects using NLTK
    related_dreams = find_related_dreams(user_input, dream_objects)

    return jsonify({"related_dreams": related_dreams})

# Pagination route that returns paginated dream objects
@app.route('/dreams', methods=['GET'])
@cross_origin()
def get_paginated_dreams():
    page = request.args.get('page', default=1, type=int)
    per_page = request.args.get('per_page', default=5, type=int)

    start_idx = (page - 1) * per_page
    end_idx = start_idx + per_page
    paginated_dreams = dream_objects[start_idx:end_idx]

    return jsonify({"dreams": paginated_dreams})

# Route that returns all dream objects paginated
@app.route('/dreams/all', methods=['GET'])
@cross_origin()
def get_all_dreams_paginated():
    page = request.args.get('page', default=1, type=int)
    per_page = request.args.get('per_page', default=5, type=int)

    start_idx = (page - 1) * per_page
    end_idx = start_idx + per_page
    paginated_dreams = dream_objects[start_idx:end_idx]

    return jsonify({"dreams": paginated_dreams})

if __name__ == '__main__':
    app.run()


Libs loaded
85 dreams loaded
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Jan/2024 12:38:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [09/Jan/2024 12:38:38] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Jan/2024 12:38:49] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:39:10] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:39:57] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:40:02] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:40:05] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:40:50] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:41:11] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:47:41] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:47:41] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:48:47] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:49:11] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 12:49:15] "GET /dreams HTTP/1.1" 200 -
127.0.0.1 - - [09/Ja